### Load Packages

In [8]:
import torch
import nbimporter
from DataLoader import load_dataset
from ConvNet import get_model
import torch.optim as optim


### Train model function

<function DataLoader.load_dataset>

### Main function

In [9]:
def main(cuda=False):
    if cuda and not torch.cuda.is_available():
        raise Exception("No GPU found")
    if cuda:
        torch.cuda.manual_seed(123)
    else:
        torch.manual_seed(123)
    
    fg_path = '/scratch/matting/dataset/Training_set/adobe/fg/'
    bg_path = '/scratch/matting/dataset/Training_set/adobe/bg/'
    alpha_path = '/scratch/matting/dataset/Training_set/adobe/alpha/'
    vgg_path = '/scratch/matting/dataset/vgg_state_dict.pth'
    fg_file = '/scratch/matting/dataset/Training_set/training_fg_names.txt'
    bg_file = '/scratch/matting/dataset/Training_set/training_bg_names.txt'
    epochs = 10
    files_list = get_files_list(fg_file, bg_file)
    dataset = load_dataset(fg_path, alpha_path, bg_path, files_list)
    model = get_model()
    vgg_dict = torch.load(vgg_path)
    model = copy_pretrain_vals(model, vgg_dict)
    if cuda:
        model = model.cuda()
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-5)
#     for i in range(epochs):
#         train_model(model, optimizer, dataset, i)
main()

### Copy pretrained model

In [4]:
def copy_pretrain_vals(model, vgg_dict):
    model.load_state_dict(vgg_dict,strict=False)
    return model

### Generating a file list array for data loader

In [3]:
def get_files_list(fg_file, bg_file):
    fg_list = []
    bg_list = []
    with open(fg_file, 'r') as f:
        fg_list = f.readlines()
    with open(bg_file, 'r') as f:
        bg_list = f.readlines()
    for i in range(len(fg_list)):
        fg_list[i] = fg_list[i].strip('\n').strip('\r')
    for i in range(len(bg_list)):
        bg_list[i] = bg_list[i].strip('\n').strip('\r')
    arr = []
    cnt = 0
    for i in range(len(fg_list)):
        for j in range(100):
            arr.append((fg_list[i], bg_list[cnt], fg_list[i]))
            cnt += 1
    return arr